In [ ]:
"""
Created on Tue Nov 05 14:56 2024

Compare depths distribution between BedMachine and eORCA1 ice draft depth for the ice shelves planned to be open in eORCA1

@author: Clara Burgard
"""

In [58]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import multimelt.create_isf_mask_functions as maskf

In [35]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [55]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
outputpath = '/home/cburgard/PLOTS/OPEN_CAVITIES/'

In [76]:
file_eORCA1 = xr.open_dataset(inputpath_raw + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc').squeeze()
file_BedMachine = xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_allvars_withcategories.nc')
file_BedMachine_masks = xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_masks_only.nc')


file_mask_eORCA1 = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1.nc').squeeze()
file_mask_BedMachine = xr.open_dataset(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc')


In [103]:
file_eORCA1['bathy_metry']

<xarray.DataArray 'bathy_metry' (y: 331, x: 360)>
[119160 values with dtype=float64]
Dimensions without coordinates: y, x

In [31]:
ID_open_list = [1,2,21,66,67,117,124,125,127,128] # probably can leave 1 and 2 closed

In [32]:
isf_draft_BM = (file_BedMachine['thickness'] - file_BedMachine['surface']).sel(category=3).drop('category')
isf_draft_NEMO = file_eORCA1['isf_draft']

In [33]:
isf_mask_BM = file_mask_BedMachine['Iceshelf_extrap'].where(isf_draft_BM > 0)
isf_mask_NEMO = file_mask_eORCA1['mask_isf']

In [83]:
for kisf in ID_open_list:
    plt.figure()
    flattened_BM_draft = isf_draft_BM.where(isf_mask_BM == kisf).values.flatten()
    flattened_BM_draft = flattened_BM_draft[~np.isnan(flattened_BM_draft)]
    
    flattened_NEMO_draft = isf_draft_NEMO.where(isf_mask_NEMO == kisf).values.flatten()
    flattened_NEMO_draft = flattened_NEMO_draft[~np.isnan(flattened_NEMO_draft)]
    
    plt.hist(flattened_BM_draft, bins=10, alpha=0.5, density=True, color='orange', label='BedMachine')
    plt.hist(flattened_NEMO_draft, bins=10, alpha=0.5, density=True, color='blue', label='eORCA1')
    plt.legend()
    plt.title(str(kisf)+' '+str(file_mask_BedMachine['NAME'].sel(ID=kisf).values))
    plt.savefig(outputpath + 'isf_draft_distrib_'+str(kisf)+'.pdf')

In [57]:
plt.close('all')

In [62]:
new_mask_NEMO = isf_mask_NEMO.where(isf_mask_NEMO != 1,134)
new_mask_BM = isf_mask_BM.where(isf_mask_BM != 1,134)

In [80]:
file_msk_NEMO = (((file_eORCA1['top_level'] == 0) * 2) + (file_eORCA1['top_level'] > 1))
file_msk_BM = file_BedMachine_masks['mask_0_1_2']

In [119]:
#new_mask : xr.DataArray
#   Array showing the coverage of each ice shelf with the respective ID, open ocean is 1, land is 0.
#file_msk : xr.DataArray
#    Mask separating ocean (0), ice shelves (between 0 and 2, excluding 0 and 2), grounded ice (2)
fronts_BM = maskf.def_ice_front(new_mask_BM, file_msk_BM)
fronts_BM = (fronts_BM + new_mask_BM).where(np.isfinite(fronts_BM))
fronts_BM = fronts_BM.where(fronts_BM != 134, 1)
fronts_NEMO = maskf.def_ice_front(new_mask_NEMO, file_msk_NEMO)
fronts_NEMO = fronts_NEMO.where(fronts_NEMO != 134, 1)


In [107]:
for kisf in ID_open_list:
    plt.figure()
    flattened_BM_draft = isf_draft_BM.where(isf_mask_BM == kisf).values.flatten()
    flattened_BM_draft = flattened_BM_draft[~np.isnan(flattened_BM_draft)]
    
    flattened_NEMO_draft = isf_draft_NEMO.where(isf_mask_NEMO == kisf).values.flatten()
    flattened_NEMO_draft = flattened_NEMO_draft[~np.isnan(flattened_NEMO_draft)]
    
    plt.hist(flattened_BM_draft, bins=10, alpha=0.5, density=True, color='orange', label='BedMachine')
    plt.axvline(x=file_BedMachine['bed'].mean('category').where(fronts_BM == kisf).max(), color='orange')
    plt.hist(flattened_NEMO_draft, bins=10, alpha=0.5, density=True, color='blue', label='eORCA1')
    plt.axvline(x=file_eORCA1['bathy_metry'].where(fronts_NEMO == kisf).max(), color='blue')
    plt.legend()
    plt.title(str(kisf)+' '+str(file_mask_BedMachine['NAME'].sel(ID=kisf).values))
    plt.savefig(outputpath + 'isf_draft_distrib_'+str(kisf)+'.pdf')

In [118]:
file_msk_BM.plot()

In [113]:
new_mask_BM.plot()

In [112]:
file_msk_BM.plot()

In [120]:
fronts_BM.plot()